In [1]:
import pandas as pd
from sqlalchemy import create_engine

def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

username = "ADMPRM$PM"
password = "Prima123Vera"
host = "localhost"
port = 1521
service_name = "XE"
dsn = f"oracle+cx_oracle://{username}:{password}@{host}:{port}/{service_name}"
engine = create_engine(dsn)

# get unique proj_id from TASK
with engine.begin() as connection:
    task_proj_id = pd.read_sql("SELECT DISTINCT proj_id FROM TASK", con=connection)

# filter PROJECT according to the result
proj_filter = task_proj_id['proj_id'].tolist()
chunked_filter = list(chunker(proj_filter, 1000))
dfs = []

for chunk in chunked_filter:
    proj_filter_string = ", ".join(map(str, chunk))
    with engine.begin() as connection:
        df_chunk = pd.read_sql(f"SELECT * FROM PROJECT WHERE proj_id IN ({proj_filter_string})", con=connection)
    dfs.append(df_chunk)

df_project = pd.concat(dfs)

# get WBS_NAME with minimum WBS_ID where proj_id of the PROJWBS table matches SUM_BASE_PROJ_ID of the filtered PROJECT table
with engine.begin() as connection:
    df_wbs = pd.read_sql("SELECT proj_id, WBS_NAME FROM (SELECT proj_id, WBS_NAME, RANK() OVER (PARTITION BY proj_id ORDER BY WBS_ID ASC) rank FROM PROJWBS) WHERE rank = 1", con=connection)

# add the resulting WBS_NAME to the PROJECT table with the name "Baseline_Project_Name"
df_project = df_project.merge(df_wbs, left_on='sum_base_proj_id', right_on='proj_id', how='left')
df_project.rename(columns={'WBS_NAME': 'Baseline_Project_Name'}, inplace=True)

# repeat the steps to add WBS_NAME as "Project_Name"
with engine.begin() as connection:
    df_wbs_project = pd.read_sql("SELECT proj_id, WBS_NAME FROM (SELECT proj_id, WBS_NAME, RANK() OVER (PARTITION BY proj_id ORDER BY WBS_ID ASC) rank FROM PROJWBS) WHERE rank = 1", con=connection)

df_project = df_project.merge(df_wbs_project, left_on='proj_id_x', right_on='proj_id', how='left')
df_project.rename(columns={'WBS_NAME': 'Project_Name'}, inplace=True)

print(df_project)

C:\Users\IEC\AppData\Local\Temp\ipykernel_17236\2253645646.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
C:\Users\IEC\AppData\Local\Temp\ipykernel_17236\2253645646.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_project = pd.concat(dfs)


      proj_id_x  fy_start_month_num chng_eff_cmp_pct_flag rsrc_self_add_flag  \
0          7670                   1                     N                  Y   
1           730                   1                     N                  Y   
2          7620                   1                     N                  Y   
3          1958                   1                     N                  Y   
4          2058                   1                     N                  Y   
...         ...                 ...                   ...                ...   
1130      30704                   1                     N                  Y   
1131      30963                   1                     N                  Y   
1132      31379                   1                     N                  Y   
1133      31382                   1                     N                  Y   
1134      32051                   1                     N                  Y   

     allow_complete_flag rsrc_multi_ass

In [3]:
df_project_list = df_project[['proj_id_x', 'proj_short_name', 'sum_base_proj_id', 'proj_id_y', 'wbs_name_x', 'proj_id', 'wbs_name_y']]

In [4]:
df_project_list

,proj_id_x,proj_short_name,sum_base_proj_id,proj_id_y,wbs_name_x,proj_id,wbs_name_y
0,7670,AR-10,NaN,NaN,NaN,7670,Jadidat Arar Land Port Backup Power Supply Pro...
1,730,B-L-H,NaN,NaN,NaN,730,Housing project baseline (PYSICAL)
2,7620,AR-7,7670.0,7670.0,Jadidat Arar Land Port Backup Power Supply Pro...,7620,Jadidat Arar Land Port Backup Power Supply Pro...
3,1958,A02,NaN,NaN,NaN,1958,WTP Baseline - Aconex rev02 submitted 21-08-20...
4,2058,A01A,NaN,NaN,NaN,2058,Aconex rev01A submitted 30-07-2022 Terminated
...,...,...,...,...,...,...,...
1130,30704,Forcast_Schedule_004-2,NaN,NaN,NaN,30704,Forcast_Schedule_004
1131,30963,1/^^f5>Dz]7N@i4nXs4GIcc)[W6VT/C9QQJ^Iw{,NaN,NaN,NaN,30963,Visual Improvement for Jadidat Arar Port Proje...
1132,31379,RfP-7 - B2,NaN,NaN,NaN,31379,Housing Baseline - B1
1133,31382,AR-45 - B1,NaN,NaN,NaN,31382,2130 Power Project Arar Recovery Schedule 27-4...


In [8]:
import pandas as pd

# File and sheet names
filename = 'Project_List.xlsx'
sheetname = 'Project_List'

# Load the sheet into a DataFrame
project_list_df = pd.read_excel(filename, sheet_name=sheetname)

In [9]:
project_list_df

,PROJECT_ID,BASELINE_PROJECT_ID,PROJECT_SHORT_NAME,BASELINE_PROJECT_NAME,PROJECT_NAME
0,7670,NaN,AR-10,NaN,Jadidat Arar Land Port Backup Power Supply Pro...
1,730,NaN,B-L-H,NaN,Housing project baseline (PYSICAL)
2,7620,7670.0,AR-7,Jadidat Arar Land Port Backup Power Supply Pro...,Jadidat Arar Land Port Backup Power Supply Pro...
3,1958,NaN,A02,NaN,WTP Baseline - Aconex rev02 submitted 21-08-20...
4,2058,NaN,A01A,NaN,Aconex rev01A submitted 30-07-2022 Terminated
...,...,...,...,...,...
1130,30704,NaN,Forcast_Schedule_004-2,NaN,Forcast_Schedule_004
1131,30963,NaN,1/^^f5>Dz]7N@i4nXs4GIcc)[W6VT/C9QQJ^Iw{,NaN,Visual Improvement for Jadidat Arar Port Proje...
1132,31379,NaN,RfP-7 - B2,NaN,Housing Baseline - B1
1133,31382,NaN,AR-45 - B1,NaN,2130 Power Project Arar Recovery Schedule 27-4...


In [11]:
import pandas as pd
import numpy as np

# Assuming df is your DataFrame
filename = 'Project_List.xlsx'
sheetname = 'Project_List'
df = pd.read_excel(filename, sheet_name=sheetname)

# Ensure BASELINE_PROJECT_ID has the same type as PROJECT_ID
df['BASELINE_PROJECT_ID'] = df['BASELINE_PROJECT_ID'].fillna(-1).astype(int)

# Prepare Update_df
Update_df = df[df['BASELINE_PROJECT_ID'] != -1]

# Prepare Baseline_Copy_df
Baseline_Copy_df = df[df['PROJECT_ID'].isin(Update_df['BASELINE_PROJECT_ID'])]

# Prepare Projects_df
Projects_df = df[~df.index.isin(Update_df.index.union(Baseline_Copy_df.index))]

In [12]:
Update_df

,PROJECT_ID,BASELINE_PROJECT_ID,PROJECT_SHORT_NAME,BASELINE_PROJECT_NAME,PROJECT_NAME
2,7620,7670,AR-7,Jadidat Arar Land Port Backup Power Supply Pro...,Jadidat Arar Land Port Backup Power Supply Pro...
5,2109,2110,A02-1,Aconex rev02 submitted 21-08-2022 Approved 24-...,Aconex rev02 submitted 21-08-2022 Approved 24-...
7,2111,2112,ARHAJJ-9,Pilgrims Baseline - B1,Update 26-10-2022
9,7671,7721,AR-11,Jadidat Arar Land Port Backup Power Supply Pro...,Jadidat Arar Land Port Backup Power Supply Pro...
11,423,526,UP-R,HIGHWAY EXPANSION BETWEEN IRAQI and SAUDI BORD...,Highway expansion between iraqi and saudi bord...
...,...,...,...,...,...
1111,26411,26461,RW-04-ALPR-RVBLLLLLL-9,Arar Land Port Fire Fighting Revised Schedule ...,Arar Land Port & Residential Area Fire Fightin...
1116,27907,27957,2059RV01-22,WTP - Revised Schedule Rev00 - Recovery 19-10-...,WTP - Revised Schedule Rev00 - Update 07-12-2023
1122,28528,28578,2057RV00A211223R00S,2057 - ARAR HIGHWAY - REVISED Rev00 - 02-11-20...,2057 - ARAR HIGHWAY - REVISED Rev00 - 02-11-20...
1123,28630,28680,19SACG00004-R2-101,Revised Schedule For Housing Project Finish da...,Housing project update 21-12-2023


In [13]:
Baseline_Copy_df

,PROJECT_ID,BASELINE_PROJECT_ID,PROJECT_SHORT_NAME,BASELINE_PROJECT_NAME,PROJECT_NAME
0,7670,-1,AR-10,NaN,Jadidat Arar Land Port Backup Power Supply Pro...
5,2109,2110,A02-1,Aconex rev02 submitted 21-08-2022 Approved 24-...,Aconex rev02 submitted 21-08-2022 Approved 24-...
6,2110,-1,A02-2,NaN,Aconex rev02 submitted 21-08-2022 Approved 24-...
8,2112,-1,ARHAJJ-9 - B1,NaN,Pilgrims Baseline - B1
10,7721,-1,AR-12,NaN,Jadidat Arar Land Port Backup Power Supply Pro...
...,...,...,...,...,...
1124,29776,-1,19SACG00004-R2-108,NaN,Revised Schedule For Housing Project Finish da...
1127,30287,-1,2057RV00A021123R00S-12,NaN,2057 - ARAR HIGHWAY - REVISED Rev00 - 02-11-20...
1128,30444,-1,Forcast_Schedule_004-1,NaN,Forcast_Schedule_004
1130,30704,-1,Forcast_Schedule_004-2,NaN,Forcast_Schedule_004


In [14]:
Projects_df

,PROJECT_ID,BASELINE_PROJECT_ID,PROJECT_SHORT_NAME,BASELINE_PROJECT_NAME,PROJECT_NAME
1,730,-1,B-L-H,NaN,Housing project baseline (PYSICAL)
3,1958,-1,A02,NaN,WTP Baseline - Aconex rev02 submitted 21-08-20...
4,2058,-1,A01A,NaN,Aconex rev01A submitted 30-07-2022 Terminated
12,3247,-1,test,NaN,test
13,523,-1,B-L-S,NaN,Improvement Works and Expansion of STP & WTP a...
...,...,...,...,...,...
1125,29878,-1,Forcast Schedule_002,NaN,Forcast Schedule_002
1126,30082,-1,Forcast_Schedule_003,NaN,Forcast_Schedule_003_11_Jan_24
1131,30963,-1,1/^^f5>Dz]7N@i4nXs4GIcc)[W6VT/C9QQJ^Iw{,NaN,Visual Improvement for Jadidat Arar Port Proje...
1132,31379,-1,RfP-7 - B2,NaN,Housing Baseline - B1


In [15]:
merged_df = pd.merge(Baseline_Copy_df, Projects_df[['PROJECT_ID', 'PROJECT_NAME']],
                     on='PROJECT_NAME', suffixes=('', '_y'))

# Rename the 'PROJECT_ID_y' column to 'Original_Baseline_Project_ID'
merged_df = merged_df.rename(columns={'PROJECT_ID_y': 'Original_Baseline_Project_ID'})

In [16]:
merged_df

,PROJECT_ID,BASELINE_PROJECT_ID,PROJECT_SHORT_NAME,BASELINE_PROJECT_NAME,PROJECT_NAME,Original_Baseline_Project_ID
0,7670,-1,AR-10,NaN,Jadidat Arar Land Port Backup Power Supply Pro...,8498
1,7670,-1,AR-10,NaN,Jadidat Arar Land Port Backup Power Supply Pro...,7003
2,7670,-1,AR-10,NaN,Jadidat Arar Land Port Backup Power Supply Pro...,8810
3,7670,-1,AR-10,NaN,Jadidat Arar Land Port Backup Power Supply Pro...,23523
4,7670,-1,AR-10,NaN,Jadidat Arar Land Port Backup Power Supply Pro...,7106
...,...,...,...,...,...,...
767,29776,-1,19SACG00004-R2-108,NaN,Revised Schedule For Housing Project Finish da...,28579
768,30287,-1,2057RV00A021123R00S-12,NaN,2057 - ARAR HIGHWAY - REVISED Rev00 - 02-11-20...,26101
769,30444,-1,Forcast_Schedule_004-1,NaN,Forcast_Schedule_004,30394
770,30704,-1,Forcast_Schedule_004-2,NaN,Forcast_Schedule_004,30394
